In [19]:
############################## can we decide between different thought categories? ############################
## Dependencies

# Import package/module for data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import modules for feature engineering and modelling
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

# pipeline
from imblearn.pipeline import Pipeline

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier

# cross validation and hyperparameter tuning
from sklearn.model_selection import StratifiedGroupKFold,  GridSearchCV

# balancing
from imblearn.over_sampling import RandomOverSampler, SMOTE

#accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

#feature selection
from sklearn import set_config
from sklearn.feature_selection import SelectKBest, chi2


In [20]:
read_path = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_labels.csv"
df = pd.read_csv(read_path)

In [21]:
df

,Unnamed: 0,Participant,Probe,Tracking Ratio [%] Mean,Fixation Duration Mean [ms],Fixation Duration Max [ms],Fixation Duration Min [ms],Fixation Duration Median [ms],Fixation Duration Std [ms],Fixation Duration Skew [ms],...,Fixation Average Pupil Diameter [mm] Kurtosis,Veregence Angles Mean [rad],Veregence Angles Std [rad],Pupil Distance Mean [px],Pupil Distance Std [px],Awareness_all,Content_cat,Content_text,Content_Probe,Awareness_all_cat
0,0,1,1,85.268,259.893077,668.0035,79.9605,246.01250,148.837048,1.275655,...,-0.397839,0.024544,0.008193,219.578560,126.804463,task-related,3.0,Lecture Comprehension (TRI),TRI,1
1,1,1,10,81.388,195.289221,738.0090,59.9435,144.00400,139.321063,2.273614,...,-0.369095,0.016063,0.008325,220.563184,106.613971,task-related,1.0,On task,On Task,1
2,2,1,11,83.450,204.240667,749.8700,63.9685,165.05425,136.716801,2.419750,...,18.112139,0.033338,0.023195,209.008402,109.885864,task-related,3.0,Lecture Comprehension (TRI),TRI,1
3,3,1,12,83.736,311.880068,1004.0200,65.9780,206.00375,266.756308,1.735261,...,13.587151,0.030035,0.018617,227.466912,125.311414,task-related,3.0,Lecture Comprehension (TRI),TRI,1
4,4,1,13,87.020,483.836531,1203.8275,111.9665,311.97925,384.029087,1.091558,...,-0.213811,0.024829,0.006304,218.641919,111.928529,task-related,2.0,Ideas about lecture (TRT),TRT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,1022,97,5,91.360,563.403233,2699.5470,107.9430,208.01900,868.559968,2.276932,...,-1.417532,0.019285,0.006715,259.802260,70.026819,aware,4.0,Personal matters (TUT),TUT,2
1023,1023,97,6,96.044,689.920744,3299.6905,85.9770,254.93450,961.214162,2.220278,...,0.946158,0.019262,0.005560,262.876279,58.771618,task-related,1.0,On task,On Task,1
1024,1024,97,7,86.734,358.844061,929.7645,101.9995,246.88975,248.337427,1.136750,...,0.018713,0.027485,0.008152,248.013878,88.994922,task-related,3.0,Lecture Comprehension (TRI),TRI,1
1025,1025,97,8,87.880,347.362106,905.9330,98.0240,256.97750,223.867392,1.267630,...,-1.078778,0.019740,0.008049,264.238181,36.298896,aware,4.0,Personal matters (TUT),TUT,2


In [22]:
# Label encoder for content probe
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df["Content_probe_categories"] = labelencoder.fit_transform(df["Content_Probe"])
df["Content_probe_categories"].unique()

# was ist der Unterschied zwischen On task and task related thougthts?

array([2, 0, 3, 4, 1])

In [23]:
#df["Awareness_all"].unique()
df["Content_probe_categories"].value_counts()

4    396
0    374
2    170
3     78
1      9
Name: Content_probe_categories, dtype: int64

In [24]:
df["Content_cat"].value_counts()

1.0     374
6.0     183
3.0     170
4.0     116
2.0      78
5.0      50
8.0      19
9.0      14
10.0     11
14.0      9
13.0      3
Name: Content_cat, dtype: int64

In [25]:
df

,Unnamed: 0,Participant,Probe,Tracking Ratio [%] Mean,Fixation Duration Mean [ms],Fixation Duration Max [ms],Fixation Duration Min [ms],Fixation Duration Median [ms],Fixation Duration Std [ms],Fixation Duration Skew [ms],...,Veregence Angles Mean [rad],Veregence Angles Std [rad],Pupil Distance Mean [px],Pupil Distance Std [px],Awareness_all,Content_cat,Content_text,Content_Probe,Awareness_all_cat,Content_probe_categories
0,0,1,1,85.268,259.893077,668.0035,79.9605,246.01250,148.837048,1.275655,...,0.024544,0.008193,219.578560,126.804463,task-related,3.0,Lecture Comprehension (TRI),TRI,1,2
1,1,1,10,81.388,195.289221,738.0090,59.9435,144.00400,139.321063,2.273614,...,0.016063,0.008325,220.563184,106.613971,task-related,1.0,On task,On Task,1,0
2,2,1,11,83.450,204.240667,749.8700,63.9685,165.05425,136.716801,2.419750,...,0.033338,0.023195,209.008402,109.885864,task-related,3.0,Lecture Comprehension (TRI),TRI,1,2
3,3,1,12,83.736,311.880068,1004.0200,65.9780,206.00375,266.756308,1.735261,...,0.030035,0.018617,227.466912,125.311414,task-related,3.0,Lecture Comprehension (TRI),TRI,1,2
4,4,1,13,87.020,483.836531,1203.8275,111.9665,311.97925,384.029087,1.091558,...,0.024829,0.006304,218.641919,111.928529,task-related,2.0,Ideas about lecture (TRT),TRT,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,1022,97,5,91.360,563.403233,2699.5470,107.9430,208.01900,868.559968,2.276932,...,0.019285,0.006715,259.802260,70.026819,aware,4.0,Personal matters (TUT),TUT,2,4
1023,1023,97,6,96.044,689.920744,3299.6905,85.9770,254.93450,961.214162,2.220278,...,0.019262,0.005560,262.876279,58.771618,task-related,1.0,On task,On Task,1,0
1024,1024,97,7,86.734,358.844061,929.7645,101.9995,246.88975,248.337427,1.136750,...,0.027485,0.008152,248.013878,88.994922,task-related,3.0,Lecture Comprehension (TRI),TRI,1,2
1025,1025,97,8,87.880,347.362106,905.9330,98.0240,256.97750,223.867392,1.267630,...,0.019740,0.008049,264.238181,36.298896,aware,4.0,Personal matters (TUT),TUT,2,4


In [7]:
# two predictions 
# 1. not so speficic: 
        # 0: on-Task,  374
        # 1: Others 9
        # 2: TRI 170
        # 3: TRT,  78
        # 4: TUT,  396


# 2. the smaller categories: 
        # 1: 'On task', 

        # 2: 'Ideas about lecture (TRT)', 

        # 3: 'Lecture Comprehension (TRI)', 

        # 4: 'Personal matters (TUT)',
        # 5: 'Daydreaming (TUT)',
        # 6: 'Current state (TUT)',
        # 8: 'Video stimulus (TUT)', 
        # 9: 'Lecture triggered (TUT)', 
        # 10:'Experiment (TUT)',
        # 13: 'Blank (TUT)']

        # 14: 'Other', 

In [8]:
# ['task-related' == 1 , 'aware' == 2, 'unaware' == 3 , nan == delete the row]
def get_X_y(train):
    FEATURES = [
        'Fixation Duration Mean [ms]', 'Fixation Duration Max [ms]', 'Fixation Duration Min [ms]', 'Fixation Duration Median [ms]', 'Fixation Duration Std [ms]', 'Fixation Duration Skew [ms]', 'Fixation Duration Quantil 25 [ms]', 'Fixation Duration Quantil 75 [ms]',
        'Saccade Duration Mean [ms]', 'Saccade Duration Max [ms]', 'Saccade Duration Min [ms]', 'Saccade Duration Median [ms]', 'Saccade Duration Std [ms]', 'Saccade Duration Skew [ms]', 'Saccade Duration Quantil 25 [ms]', 'Saccade Duration Quantil 75 [ms]', 
        'Blink Duration Mean [ms]', 'Blink Duration Max [ms]', 'Blink Duration Min [ms]', 'Blink Duration Median [ms]', 'Blink Duration Std [ms]', 'Blink Duration Skew [ms]', 'Blink Duration Quantil 25 [ms]', 'Blink Duration Quantil 75 [ms]', 'Fixation Duration Kurtosis [ms]',
        'Saccade Duration Kurtosis [ms]',
        'Blink Duration Kurtosis [ms]', 
        'Fixation Saccade Ratio Mean', 'Fixation Saccade Ratio Max', 'Fixation Saccade Ratio Min', 'Fixation Saccade Ratio Median', 'Fixation Saccade Ratio Std', 'Fixation Saccade Ratio Skew', 'Fixation Saccade Ratio Kurtosis', 
        'Fixation Number', 'Blink Number', 
        'Fixation Dispersion X Mean [px]', 'Fixation Dispersion X Max [px]', 'Fixation Dispersion X Min [px]', 'Fixation Dispersion X Median [px]', 'Fixation Dispersion X Std [px]', 'Fixation Dispersion X Skew [px]', 'Fixation Dispersion X Quantil 25 [px]', 'Fixation Dispersion X Quantil 75 [px]', 
        'Fixation Dispersion Y Mean [px]', 'Fixation Dispersion Y Max [px]', 'Fixation Dispersion Y Min [px]', 'Fixation Dispersion Y Median [px]', 'Fixation Dispersion Y Std [px]', 'Fixation Dispersion Y Skew [px]', 'Fixation Dispersion Y Quantil 25 [px]', 'Fixation Dispersion Y Quantil 75 [px]', 'Fixation Dispersion X Kurtosis [px]', 'Fixation Dispersion Y Kurtosis [px]', 
        'Saccade Amplitude Mean [°]', 'Saccade Amplitude Max [°]', 'Saccade Amplitude Min [°]', 'Saccade Amplitude Median [°]', 'Saccade Amplitude Std [°]', 'Saccade Amplitude Skew [°]', 'Saccade Amplitude Quantil 25 [°]', 'Saccade Amplitude Quantil 75 [°]', 'Saccade Amplitude Kurtosis [°]',
        'Saccade Acceleration Average [°/s²] Mean', 'Saccade Acceleration Average [°/s²] Max', 'Saccade Acceleration Average [°/s²] Min', 'Saccade Acceleration Average [°/s²] Median', 'Saccade Acceleration Average [°/s²] Std', 'Saccade Acceleration Average [°/s²] Skew]', 'Saccade Acceleration Average [°/s²] Quantil 25]', 'Saccade Acceleration Average [°/s²] Quantil 75]',
        'Saccade Acceleration Peak [°/s²] Mean', 'Saccade Acceleration Peak [°/s²] Max', 'Saccade Acceleration Peak [°/s²] Min', 'Saccade Acceleration Peak [°/s²] Median', 'Saccade Acceleration Peak [°/s²] Std', 'Saccade Acceleration Peak [°/s²] Skew]', 'Saccade Acceleration Peak [°/s²] Quantil 25]', 'Saccade Acceleration Peak [°/s²] Quantil 75]', 'Saccade Deceleration Peak [°/s²] Mean', 
        'Saccade Deceleration Peak [°/s²] Max', 'Saccade Deceleration Peak [°/s²] Min', 'Saccade Deceleration Peak [°/s²] Median', 'Saccade Deceleration Peak [°/s²] Std', 'Saccade Deceleration Peak [°/s²] Skew]', 'Saccade Deceleration Peak [°/s²] Quantil 25]', 'Saccade Deceleration Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Average [°/s²] Mean', 'Saccade Velocity Average [°/s²] Max', 'Saccade Velocity Average [°/s²] Min', 'Saccade Velocity Average [°/s²] Median', 'Saccade Velocity Average [°/s²] Std', 'Saccade Velocity Average [°/s²] Skew]', 'Saccade Velocity Average [°/s²] Quantil 25]', 'Saccade Velocity Average [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [°/s²] Mean', 'Saccade Velocity Peak [°/s²] Max', 'Saccade Velocity Peak [°/s²] Min', 'Saccade Velocity Peak [°/s²] Median', 'Saccade Velocity Peak [°/s²] Std', 'Saccade Velocity Peak [°/s²] Skew]', 'Saccade Velocity Peak [°/s²] Quantil 25]', 'Saccade Velocity Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [%] Mean', 'Saccade Velocity Peak [%] Max', 'Saccade Velocity Peak [%] Min', 'Saccade Velocity Peak [%] Median', 'Saccade Velocity Peak [%] Std', 'Saccade Velocity Peak [%] Skew]', 'Saccade Velocity Peak [%] Quantil 25]', 'Saccade Velocity Peak [%] Quantil 75]', 
        'Saccade Acceleration Average [°/s²] Kurtosis', 'Saccade Acceleration Peak [°/s²] Kurtosis', 'Saccade Deceleration Peak [°/s²] Kurtosis', 'Saccade Velocity Average [°/s²] Kurtosis', 'Saccade Velocity Peak [°/s²] Kurtosis', 'Saccade Velocity Peak [%] Kurtosis', 
        'Saccade Length Mean [px]', 'Saccade Length Max [px]', 'Saccade Length Min [px]', 'Saccade Length Median [px]', 'Saccade Length Std [px]', 'Saccade Length Skew [px]]', 'Saccade Length Quantil 25 [px]]', 'Saccade Length Quantil 75 [px]]', 'Saccade Length Kurtosis [px]', 
        'Fixation Average Pupil Diameter [mm] Mean', 'Fixation Average Pupil Diameter [mm] Max', 'Fixation Average Pupil Diameter [mm] Min', 'Fixation Average Pupil Diameter [mm] Median', 'Fixation Average Pupil Diameter [mm] Std', 'Fixation Average Pupil Diameter [mm] Skew', 'Fixation Average Pupil Diameter [mm] Quantil25', 'Fixation Average Pupil Diameter [mm] Quantil75',
        'Fixation Average Pupil Diameter [mm] Kurtosis', 
        'Veregence Angles Mean [rad]', 'Veregence Angles Std [rad]', 
        'Pupil Distance Mean [px]', 'Pupil Distance Std [px]'
    ]

    GROUPS = "Participant"

    TARGET_THOUGHT_CAT = "Content_probe_categories"
    TARGET_SPECIFIC_THOUGHT_CAT = "Content_cat"
    
    X = train[FEATURES]
    y_thoughts_cat = train[TARGET_THOUGHT_CAT]
    y_specific_thought_cat = train[TARGET_SPECIFIC_THOUGHT_CAT]

    groups = train[GROUPS]

    return X, y_thoughts_cat,y_specific_thought_cat, groups

In [9]:
X, y_thoughts_cat,y_specific_thought_cat, groups = get_X_y(df)

In [10]:
#### Thought Categories not specific
## Pipleline
imputer =  SimpleImputer(fill_value='missing')
scaler = StandardScaler()
over = SMOTE(random_state= 27) 

model = MLPClassifier()

# getuned
#model = RandomForestClassifier(random_state=0,bootstrap= True, max_depth = 50, max_features="auto", min_samples_leaf = 10, min_samples_split= 1, n_estimators = 50)

# evtl. SVC noch mehr tunen
#model = SVC(C = 0.1, gamma = 0.5, kernel = "linear")

#model = xgb.XGBClassifier(objective="objective=multi:softmax", random_state=42, colsample_bytree = 1, max_depth = 3, n_estimators = 100, subsample = 1)
#model = GaussianNB()

steps = [('imputer', imputer), ('scaler',scaler),('over', over), ('model', model)]
pipe = Pipeline(steps=steps)

In [11]:
##################### prediction without baseline #####################
import random
from sklearn import metrics
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0

f1_scores ={"0": [], "1": [], "2": [], "3": [], "4": []}
precision_scores = {"0": [], "1": [], "2": [], "3": [], "4": []}
recall_scores ={"0": [], "1": [], "2": [], "3": [], "4": []}

### stratifies group k fold
for train_index, test_index in sgk.split(X, y_thoughts_cat, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y_thoughts_cat.loc[train_index], y_thoughts_cat.loc[test_index]

    # Fit Model on Train
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    #accurancy_score = accuracy_score(y_test, y_pred)

    labels = [0, 1, 2, 3, 4]
    f1 = f1_score(y_test, y_pred, average=None, labels=labels)
    f1_scores["0"].append(f1[0])
    f1_scores["1"].append(f1[1])
    f1_scores["2"].append(f1[2])
    f1_scores["3"].append(f1[3])
    f1_scores["4"].append(f1[4])


    #print(f1)

    recall = recall_score(y_test, y_pred, average=None, labels=labels)
    recall_scores["0"].append(recall[0])
    recall_scores["1"].append(recall[1])
    recall_scores["2"].append(recall[2])
    recall_scores["3"].append(recall[3])
    recall_scores["4"].append(recall[4])

    #print(recall)

    precision = precision_score(y_test, y_pred, average=None, labels=labels)
    precision_scores["0"].append(precision[0])
    precision_scores["1"].append(precision[1])
    precision_scores["2"].append(precision[2])
    precision_scores["3"].append(precision[3])
    precision_scores["4"].append(precision[4])

    #print(recall)

    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
    #auc = metrics.auc(fpr, tpr)

    #print(f"f1 with labels  score for fold {fold}: ", auc)
    print(f"f1 with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, f1)})
    print(f"recall with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, recall)})
    print(f"precision with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, precision)})
    
    fold += 1


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


f1 with labels  score for fold 0:  {0: 0.48648648648648646, 1: 0.0, 2: 0.1, 3: 0.14285714285714288, 4: 0.44000000000000006}
recall with labels  score for fold 0:  {0: 0.5142857142857142, 1: 0.0, 2: 0.075, 3: 0.13333333333333333, 4: 0.4782608695652174}
precision with labels  score for fold 0:  {0: 0.46153846153846156, 1: 0.0, 2: 0.15, 3: 0.15384615384615385, 4: 0.4074074074074074}


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


f1 with labels  score for fold 1:  {0: 0.4689655172413793, 1: 0.0, 2: 0.24489795918367346, 3: 0.12121212121212122, 4: 0.44155844155844154}
recall with labels  score for fold 1:  {0: 0.4722222222222222, 1: 0.0, 2: 0.24, 3: 0.08695652173913043, 4: 0.4788732394366197}
precision with labels  score for fold 1:  {0: 0.4657534246575342, 1: 0.0, 2: 0.25, 3: 0.2, 4: 0.40963855421686746}


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


f1 with labels  score for fold 2:  {0: 0.41666666666666663, 1: 0.0, 2: 0.19753086419753083, 3: 0.0, 4: 0.45454545454545453}
recall with labels  score for fold 2:  {0: 0.43478260869565216, 1: 0.0, 2: 0.20512820512820512, 3: 0.0, 4: 0.4411764705882353}
precision with labels  score for fold 2:  {0: 0.4, 1: 0.0, 2: 0.19047619047619047, 3: 0.0, 4: 0.46875}


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


f1 with labels  score for fold 3:  {0: 0.463768115942029, 1: 0.0, 2: 0.28571428571428575, 3: 0.07407407407407407, 4: 0.5371428571428571}
recall with labels  score for fold 3:  {0: 0.4266666666666667, 1: 0.0, 2: 0.3142857142857143, 3: 0.08333333333333333, 4: 0.5465116279069767}
precision with labels  score for fold 3:  {0: 0.5079365079365079, 1: 0.0, 2: 0.2619047619047619, 3: 0.06666666666666667, 4: 0.5280898876404494}
f1 with labels  score for fold 4:  {0: 0.3972602739726027, 1: 0.0, 2: 0.1111111111111111, 3: 0.0625, 4: 0.48}
recall with labels  score for fold 4:  {0: 0.32954545454545453, 1: 0.0, 2: 0.0967741935483871, 3: 0.05555555555555555, 4: 0.6176470588235294}
precision with labels  score for fold 4:  {0: 0.5, 1: 0.0, 2: 0.13043478260869565, 3: 0.07142857142857142, 4: 0.3925233644859813}


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
# f1 score
#  # 0: on-Task,  374
        # 1: Others 9
        # 2: TRI 170
        # 3: TRT,  78
        # 4: TUT,  396


f1_averages = [(k, sum(v)/len(v)) for k, v in f1_scores.items()]
print("f1 score:",f1_averages)



f1 score: [('0', 0.4466294120618328), ('1', 0.0), ('2', 0.18785084404132024), ('3', 0.08012866762866763), ('4', 0.47064935064935065)]


In [107]:
#precision
precision_averages = [(k, sum(v)/len(v)) for k, v in precision_scores.items()]
print("precision:", precision_averages)
# for k, v in averages:
#     print("f1 score, label", k, f"\n{v}")

# [('0', 0.44543620918089), ('1', 0.0), ('2', 0.2274671964033666), ('3', 0.05179615705931495), ('4', 0.5063787833974168)]

[('0', 0.44543620918089), ('1', 0.0), ('2', 0.2274671964033666), ('3', 0.05179615705931495), ('4', 0.5063787833974168)]


In [108]:
# recall
recall_averages = [(k, sum(v)/len(v)) for k, v in recall_scores.items()]
print("recall:", recall_averages)
# for k, v in averages:
#     print("f1 score, label", k, f"\n{v}")

#[('0', 0.49935491561578516), ('1', 0.0), ('2', 0.2892644452321872), ('3', 0.05555555555555556), ('4', 0.3907504225575503)]

[('0', 0.49935491561578516), ('1', 0.0), ('2', 0.2892644452321872), ('3', 0.05555555555555556), ('4', 0.3907504225575503)]


In [109]:
##################### prediction without baseline with macro f1 score #####################
import random
from sklearn import metrics
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0

f1_scores =[]
precision_scores = []
recall_scores =[]

### stratifies group k fold
for train_index, test_index in sgk.split(X, y_thoughts_cat, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y_thoughts_cat.loc[train_index], y_thoughts_cat.loc[test_index]

    # Fit Model on Train
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    #accurancy_score = accuracy_score(y_test, y_pred)

    labels = [0, 1, 2, 3, 4]
    f1 = f1_score(y_test, y_pred, average= "macro", labels=labels)
    f1_scores.append(f1)
  
    #print(f1)

    recall = recall_score(y_test, y_pred, average= "macro", labels=labels)
    recall_scores.append(recall)

    #print(recall)

    precision = precision_score(y_test, y_pred, average="macro", labels=labels)
    precision_scores.append(precision)

    #print(recall)

    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
    #auc = metrics.auc(fpr, tpr)

    fold += 1


mean_f1 = np.mean(f1_scores)
mean_recall = np.mean(recall_scores)
mean_precision = np.mean(precision_scores)

print(f'Our mean fold f1 score is {mean_f1:0.4f}')
print(f'Our mean fold recall is {mean_recall:0.4f}')
print(f'Our mean fold precision is {mean_precision:0.4f}')


# Our mean fold f1 score is 0.2419
# Our mean fold recall is 0.2470
# Our mean fold precision is 0.2462

C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\lo

Our mean fold f1 score is 0.2419
Our mean fold recall is 0.2470
Our mean fold precision is 0.2462


In [56]:
###### Hyperparameter tuning with gridsearch
# # max_depth: 3–10
# n_estimators: 100 (lots of observations) to 1000 (few observations)
# learning_rate: 0.01–0.3
# colsample_bytree: 0.5–1
# subsample: 0.6–1

# XGBoost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# defining parameter range
param_grid = {
    "model__subsample": [0.9, 1], #np.arange(0.6,1,0.05),
    "model__max_depth": [3, 6, 10], # np.arange(3,10,1),
    "model__n_estimators": [150, 100, 80],
    "model__colsample_bytree": [0.8, 1.0]
}

grid = GridSearchCV(pipe, param_grid = param_grid, verbose = 1, cv = 5, scoring= "f1_macro")

# muss hier mit trainig data oder mit ganzem X trainiert werden?
grid.fit(X, y_thoughts_cat)

print(grid.best_params_)
print(grid.best_estimator_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'model__colsample_bytree': 0.8, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 1}
Pipeline(steps=[('imputer', SimpleImputer(fill_value='missing')),
                ('scaler', StandardScaler()), ('over', SMOTE(random_state=27)),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.8, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_s

In [18]:
###### Hyperparameter tuning with gridsearch
# Naive Bayes
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Gaussian
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)
# defining parameter range

param_grid =  {'model__var_smoothing': np.logspace(0,-9, num=100)}
grid = GridSearchCV(pipe, param_grid = param_grid, verbose = 1, cv = 5, scoring= "f1_macro")

grid.fit(X, y_thoughts_cat)
 
print(grid.best_params_)
print(grid.best_estimator_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'model__var_smoothing': 0.003511191734215131}
Pipeline(steps=[('imputer', SimpleImputer(fill_value='missing')),
                ('scaler', StandardScaler()), ('over', SMOTE(random_state=27)),
                ('model', GaussianNB(var_smoothing=0.003511191734215131))])


In [77]:
###### Hyperparameter tuning with gridsearch
# SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# {'model__C': 1, 'model__gamma': 0.005, 'model__kernel': 'linear'}

# SVC
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)
# defining parameter range
param_grid = {'model__C': [0.1, 1, 5], 
              'model__gamma':  [0, 1, 0.5],
              'model__kernel': ["linear", "rbf"]} 
  
grid = GridSearchCV(pipe, param_grid = param_grid, verbose = 1, cv = 5, scoring= "f1_macro")

grid.fit(X, y_thoughts_cat)
 
print(grid.best_params_)
print(grid.best_estimator_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[11:32:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "C", "kernel" } are not used.

[11:32:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "C", "kernel" } are not used.

[11:32:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "C", "kernel" } are not used.

[11:32:13] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "C", "kernel" } are not used.

[11:32:14] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/

In [110]:
# random Forrest

###### Grid Search random forrest
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)
# defining parameter range
param_grid = {
    'model__bootstrap': [True, False],
 'model__max_depth': [50, 10, 6],
 'model__max_features': ['auto'],
 'model__min_samples_leaf': [5, 10, 15],
 'model__min_samples_split': [1,2],
 'model__n_estimators': [50, 100, 70]}

grid = GridSearchCV(pipe, param_grid = param_grid, verbose = 1, cv = 5, scoring= "f1_macro")

grid.fit(X, y_thoughts_cat)
 
print(grid.best_params_)
print(grid.best_estimator_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\lo

{'model__bootstrap': True, 'model__max_depth': 50, 'model__max_features': 'auto', 'model__min_samples_leaf': 10, 'model__min_samples_split': 1, 'model__n_estimators': 50}
Pipeline(steps=[('imputer', SimpleImputer(fill_value='missing')),
                ('scaler', StandardScaler()), ('over', SMOTE(random_state=27)),
                ('model',
                 RandomForestClassifier(max_depth=50, max_features='auto',
                                        min_samples_leaf=10,
                                        min_samples_split=1, n_estimators=50,
                                        random_state=0))])


In [ ]:
##################### prediction with baseline #####################
# distribution:
        # 4    396
        # 0    374
        # 2    170
        # 3     78
        # 1      9

# 1. not so speficic: 
        # 0: on-Task, 36.42 %
        # 1: Others
        # 2: TRI 16.55%
        # 3: TRT, 7.59%
        # 4: TUT,  38.55%
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
f1_scores ={"0": [], "1": [], "2": [], "3": [], "4": []}
precision_scores = {"0": [], "1": [], "2": [], "3": [], "4": []}
recall_scores ={"0": [], "1": [], "2": [], "3": [], "4": []}

### stratifies group k fold
for i in range(10):
    for train_index, test_index in sgk.split(X, y_thoughts_cat, groups):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y_thoughts_cat.loc[train_index], y_thoughts_cat.loc[test_index]
    
        # Fit Model on Train
        pipe.fit(X_train, y_train)
    
        y_pred = pipe.predict(X_test)
    
        # create baseline
        baseline = np.ones(len(y_pred))
        TUT_size = 396/1027 * len(y_pred)
        baseline[: int(TUT_size)] = 4

        TRT_size =  78/1027 * len(y_pred)
        baseline[int(TUT_size):int(TRT_size)] = 3

        
        TRI_size =  170/1027 * len(y_pred)
        baseline[int(TRT_size):int(TRI_size)] = 2

        On_Task_size =  374/1027 * len(y_pred)
        baseline[int(TRI_size):int(On_Task_size)] = 0

        np.random.shuffle(baseline)
        baseline = baseline.astype(int)
    
        #accurancy_score = accuracy_score(y_test, y_pred)
    
        labels = [0, 1, 2, 3, 4]
        f1 = f1_score(y_test, baseline, average=None, labels=labels)
        f1_scores["0"].append(f1[0])
        f1_scores["1"].append(f1[1])
        f1_scores["2"].append(f1[2])
        f1_scores["3"].append(f1[3])
        f1_scores["4"].append(f1[4])


    #print(f1)

        recall = recall_score(y_test, baseline, average=None, labels=labels)
        recall_scores["0"].append(recall[0])
        recall_scores["1"].append(recall[1])
        recall_scores["2"].append(recall[2])
        recall_scores["3"].append(recall[3])
        recall_scores["4"].append(recall[4])

    #print(recall)

        precision = precision_score(y_test, baseline, average=None, labels=labels)
        precision_scores["0"].append(precision[0])
        precision_scores["1"].append(precision[1])
        precision_scores["2"].append(precision[2])
        precision_scores["3"].append(precision[3])
        precision_scores["4"].append(precision[4])

    #print(recall)
        #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
        #auc = metrics.auc(fpr, tpr)
    
        #print(f"f1 with labels  score for fold {fold}: ", auc)
        print(f"f1 with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, f1)})
        print(f"recall with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, recall)})
        print(f"precision with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, precision)})
        
        fold += 1

In [114]:
averages = [(k, sum(v)/len(v)) for k, v in f1_scores.items()]
averages
# for k, v in averages:
#     print("f1 score, label", k, f"\n{v}")

[('0', 0.24975545172452637),
 ('1', 0.015774399302673284),
 ('2', 0.11302282213749194),
 ('3', 0.0),
 ('4', 0.15079996769765003)]

In [37]:
averages = [(k, sum(v)/len(v)) for k, v in precision_scores.items()]
for k, v in averages:
    print(" presicion score, label", k, f"\n{v}")

 presicion score, label 0 
0.3635291480734356
 presicion score, label 1 
0.007694427517336508
 presicion score, label 2 
0.16309253525971792
 presicion score, label 3 
0.0
 presicion score, label 4 
0.3771929824561403


In [38]:
averages = [(k, sum(v)/len(v)) for k, v in recall_scores.items()]
for k, v in averages:
    print("recall score, label", k, f"\n{v}")

recall score, label 0 
0.19783499592195244
recall score, label 1 
0.5333333333333333
recall score, label 2 
0.09761963842609005
recall score, label 3 
0.0
recall score, label 4 
0.08953667084686592


In [39]:
##################### prediction with baseline #####################
# distribution:
        # 4    396
        # 0    374
        # 2    170
        # 3     78
        # 1      9

# 1. not so speficic: 
        # 0: on-Task, 
        # 1: Others
        # 2: TRI
        # 3: TRT, 
        # 4: TUT,  
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
f1_scores =[]
precision_scores = []
recall_scores =[]

### stratifies group k fold
for i in range(10):
    for train_index, test_index in sgk.split(X, y_thoughts_cat, groups):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y_thoughts_cat.loc[train_index], y_thoughts_cat.loc[test_index]
    
        # Fit Model on Train
        pipe.fit(X_train, y_train)
    
        y_pred = pipe.predict(X_test)
    
        # create baseline
        baseline = np.ones(len(y_pred))
        TUT_size = 396/1027 * len(y_pred)
        baseline[: int(TUT_size)] = 4

        TRT_size =  78/1027 * len(y_pred)
        baseline[int(TUT_size):int(TRT_size)] = 3

        TRI_size =  170/1027 * len(y_pred)
        baseline[int(TRT_size):int(TRI_size)] = 2

        On_Task_size =  374/1027 * len(y_pred)
        baseline[int(TRI_size):int(On_Task_size)] = 0

        np.random.shuffle(baseline)
        baseline = baseline.astype(int)
    
    
        labels = [0, 1, 2, 3, 4]
        f1 = f1_score(y_test, baseline, average= "macro", labels=labels)
        f1_scores.append(f1)
    

        recall = recall_score(y_test, baseline, average= "macro", labels=labels)
        recall_scores.append(recall)


        precision = precision_score(y_test, baseline, average="macro", labels=labels)
        precision_scores.append(precision)


        #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
        #auc = metrics.auc(fpr, tpr)
    
        print(f"f1  score for fold {fold}: ", f1)
        print(f"recall for fold {fold}: ", recall)
        print(f"precision for fold {fold}: ", precision)
        
        fold += 1

mean_f1 = np.mean(f1_scores)
mean_recall = np.mean(recall_scores)
mean_precision = np.mean(precision_scores)

print(f'Our mean fold f1 score is {mean_f1:0.4f}')
print(f'Our mean fold recall is {mean_recall:0.4f}')
print(f'Our mean fold precision is {mean_precision:0.4f}')



C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: 

f1  score for fold 0:  0.0992880692112811
recall for fold 0:  0.2695341614906832
precision for fold 0:  0.1683333333333333
f1  score for fold 1:  0.11395297512944572
recall for fold 1:  0.2826071987480438
precision for fold 1:  0.18435745395392766
f1  score for fold 2:  0.11143120610936572
recall for fold 2:  0.21180405272476882
precision for fold 2:  0.19956521739130434


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1  score for fold 3:  0.10302216302216301
recall for fold 3:  0.1726998892580288
precision for fold 3:  0.18299209562367455
f1  score for fold 4:  0.10615705385232248
recall for fold 4:  0.2739347938588926
precision for fold 4:  0.16538430680359434


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: 

...

C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: 

f1  score for fold 43:  0.11096681096681098
recall for fold 43:  0.18058028792912514
precision for fold 43:  0.17923269712743398
f1  score for fold 44:  0.10206863652252587
recall for fold 44:  0.16928583750215628
precision for fold 44:  0.18487443838254172
f1  score for fold 45:  0.11356625122119357
recall for fold 45:  0.2801656314699793
precision for fold 45:  0.18628205128205128


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: 

f1  score for fold 46:  0.10160088277735338
recall for fold 46:  0.271868544600939
precision for fold 46:  0.17259274807157474
f1  score for fold 47:  0.10066700747719208
recall for fold 47:  0.20243950422978557
precision for fold 47:  0.18734299516908212
f1  score for fold 48:  0.14611758611758613
recall for fold 48:  0.30427906976744185
precision for fold 48:  0.23615962984384037
f1  score for fold 49:  0.1417432607488742
recall for fold 49:  0.29803346558564775
precision for fold 49:  0.23906851732991016
Our mean fold f1 score is 0.1083
Our mean fold recall is 0.2066
Our mean fold precision is 0.1864


C:\Users\Julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
